In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import mahotas as mh
import numpy as np

In [3]:
DATA_DIR = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/classified_images/good_images/'
good ='/raid/mpsych/RISTERLAB/fly_segmentation_experiments/classified_images/good_7/'
bad ='/raid/mpsych/RISTERLAB/fly_segmentation_experiments/classified_images/bad_images_1/'

In [4]:
img_file_names = os.listdir(DATA_DIR)

In [5]:
img_file_names_sorted = sorted(img_file_names)

In [6]:
len(os.listdir(DATA_DIR))

254

In [7]:
img_file_names_sorted

['.ipynb_checkpoints',
 '000000_000000_000008_000000-Copy1.tif',
 '000000_000000_000008_000000.tif',
 '000000_000000_000031_000000-Copy1.tif',
 '000000_000000_000031_000000.tif',
 '000000_000000_000043_000000-Copy1.tif',
 '000000_000000_000043_000000.tif',
 '000000_000000_000078_000000-Copy1.tif',
 '000000_000000_000078_000000.tif',
 '000000_000000_000085_000000-Copy1.tif',
 '000000_000000_000085_000000.tif',
 '000000_000000_000170_000000-Copy1.tif',
 '000000_000000_000170_000000.tif',
 '000000_000000_000173_000000-Copy1.tif',
 '000000_000000_000173_000000.tif',
 '000000_000000_000174_000000-Copy1.tif',
 '000000_000000_000174_000000.tif',
 '000000_000000_000175_000000-Copy1.tif',
 '000000_000000_000175_000000.tif',
 '000000_000000_000180_000000-Copy1.tif',
 '000000_000000_000180_000000.tif',
 '000000_000000_000181_000000.tif',
 '000000_000000_000182_000000.tif',
 '000000_000000_000183_000000.tif',
 '000000_000000_000184_000000.tif',
 '000000_000000_000185_000000.tif',
 '000000_000000_0

In [8]:
# loading images
slices = []
slice_address = []
for z in img_file_names_sorted:
    if (z.endswith("tif")): # checking the file ends with tif
        img = z
        slice_address.append(img)
        z_img = mh.imread(os.path.join(DATA_DIR,img))
        z_img_cropped = z_img[1500:2000, 2500:4000].astype(np.float)
        z_img_cropped /= z_img_cropped.max()
        z_img_cropped *= 255
        

        slices.append(z_img_cropped)

/tmp/ipykernel_948303/1614629096.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z_img_cropped = z_img[1500:2000, 2500:4000].astype(np.float)


In [9]:
len(slices)

253

In [10]:
# plotting, binary masking and classifying as good and bad images
    
i = 0
j = 0
address_index = 0
for s in slices:
    # apply gaussian filter
    b = s.copy()
    b = mh.gaussian_filter(b, sigma=3)
    
    # filter by value
    b[b<100] = 0 # Could be played around with later
    
    # label regions
    labeled, number  = mh.label(b)
    #print("before size red {}".format(number))
    
    # filter based on labeled region size
    sizes = mh.labeled.labeled_size(labeled)
    too_small = np.where(sizes < 1000)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)
    # print(" after red {}".format(labeled_only_big))
    # create binary mask - foreground and background
    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1
    # print("After removing small islands {}".format(number_1))
   
    # close holes in binary mask
    binary_mask_closed_holes = mh.morph.close_holes(binary_mask)
    
    # close region in binary mask
    binary_mask_closed = mh.morph.close(binary_mask_closed_holes)
    
    # dilate and erode to fill further holes
    for i in range(4):
        binary_mask_closed = mh.morph.dilate(binary_mask_closed)
    for i in range(4):
        binary_mask_closed = mh.morph.erode(binary_mask_closed)
    
    # apply gaussian filter on close region binary mask
    binary_mask_closed_filtered = mh.gaussian_filter(binary_mask_closed, sigma=3)
    
    # label regions
    labeled_mask, number_1  = mh.label(binary_mask_closed)
    
    # filter based on labeled region size
    sizes = mh.labeled.labeled_size(labeled_mask)
    too_small = np.where(sizes < 1000)
    labeled_mask_only_big = mh.labeled.remove_regions(labeled_mask, too_small)
    
    # create binary mask - foreground and background
    final_binary_mask = labeled_mask_only_big.copy()
    final_binary_mask[final_binary_mask > 0] = 1
    
    
    if number_1 <=50 and number_1 >=5:
        #print(number_1)
        print(slice_address[address_index])
        mh.imsave(os.path.join(good, f"{slice_address[address_index].split('.')[0]}.tif"), final_binary_mask)
        j += 1
        #shutil.copy((DATA_DIR + '/' + slice_address[address_index]), (good + '/' + slice_address[address_index]))
    else:
        #print(number_1)
        #shutil.copy((DATA_DIR + '/' + slice_address[address_index]), (bad + '/' + slice_address[address_index]))

        i += 1
    
    
    address_index+=1

000000_000000_000008_000000-Copy1.tif
000000_000000_000008_000000.tif
000000_000000_000031_000000-Copy1.tif
000000_000000_000031_000000.tif
000000_000000_000043_000000-Copy1.tif
000000_000000_000043_000000.tif
000000_000000_000078_000000-Copy1.tif
000000_000000_000078_000000.tif
000000_000000_000085_000000-Copy1.tif
000000_000000_000085_000000.tif
000000_000000_000170_000000-Copy1.tif
000000_000000_000170_000000.tif
000000_000000_000173_000000-Copy1.tif
000000_000000_000173_000000.tif
000000_000000_000174_000000-Copy1.tif
000000_000000_000174_000000.tif
000000_000000_000175_000000-Copy1.tif
000000_000000_000175_000000.tif
000000_000000_000180_000000-Copy1.tif
000000_000000_000180_000000.tif
000000_000000_000181_000000.tif
000000_000000_000182_000000.tif
000000_000000_000183_000000.tif
000000_000000_000184_000000.tif
000000_000000_000185_000000.tif
000000_000000_000187_000000.tif
000000_000000_000189_000000.tif
000000_000000_000190_000000.tif
000000_000000_000191_000000.tif
000000_00000

In [11]:
# count = 0
# for path in os.scandir(bad):
#     if path.is_file():
#         count += 1
# print('file count:', count)
# subplot_value = int(count) 

In [12]:
# plt.figure()
# f, axarr = plt.subplots((subplot_value), 2, figsize=(20, 40))

# i = 0
# address_index = 0
# for s in slices:
#     if number_1 <=25 and number_1 >=50:
#         axarr[i,0].set_title("{}  {}".format(number_1, slice_address[address_index]))
#         axarr[i,0].imshow(s)
#         axarr[i,1].set_title("{}  {}".format(number_1, slice_address[address_index]))
#         axarr[i,1].imshow(binary_mask_closed_filtered, 'jet', interpolation='none', alpha=0.7)
#         i += 1
        
#     address_index+=1
        